Numpy 버전 issue로 1.23으로 downgrade\
참고: https://github.com/NVIDIA/TensorRT/issues/2567

In [ ]:
# requirement 다운로드
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install numpy==1.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.4 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661749 sha256=dfb1eea3e77c2bd6b2d0784037b9dbfe4be6096c679fff45d57eafdb4ec71c40
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86

Azure에서 serving하던 모델 다운로드 서비스 문제로 hugging face를 통한 모델 다운로드 버전의 코드 변경\
참고1: https://github.com/SKTBrain/KoBERT/issues/102 \
참고2: https://github.com/SKTBrain/KoBERT/issues/67 \
아래에 있는 KoBERT 관련 코드들도 이에 맞게 수정

In [ ]:
# colab과 google drive 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# KoBERT를 github에서 colab으로 다운로드

# 변경 전)
# !pip install git+https://github.com/SKTBrain/KoBERT.git@master

# 변경 후)
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-ordek7e3/kobert-tokenizer_2abd18ee886a49abb2ccf349bc68c9a1
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-ordek7e3/kobert-tokenizer_2abd18ee886a49abb2ccf349bc68c9a1
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=0dc1972dbb54555e1f298924b61b9a1f320808857c606ac2107555b670d58aff
  Stored in directory: /tmp/pip-ephem-wheel-cache-eapb4hgu/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
# KoBERT 관련 library import

# 변경 전)
# from kobert.utils import get_tokenizer
# from kobert.pytorch_kobert import get_pytorch_kobert_model

# 변경 후)
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [ ]:
# Transformers 관련 libary import
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
print(torch.cuda.is_available())

False


In [ ]:
# colab GPU 사용, 실패 시 CPU로 대체
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [ ]:
# 관련 데이터를 save/load할 path
myDrivePath = '/MyDrive/2024-1/AI 프로젝트 입문/colab/'

In [ ]:
# BERTSentenceTransform를 변경된 KoBERT tokenizer에 맞게 수정
# docs: https://nlp.gluon.ai/api/data.html#gluonnlp.data.BERTSentenceTransform
# 원래 코드: https://nlp.gluon.ai/_modules/gluonnlp/data/transforms.html#BERTSentenceTransform
class KoBERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    vocab : Vocab
        The vocabulary which has cls_token and sep_token registered.
        If vocab.cls_token is not present, vocab.bos_token is used instead.
        If vocab.sep_token is not present, vocab.eos_token is used instead.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length, vocab=None, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = self._tokenizer.vocab if vocab is None else vocab
        # RoBERTa does not register CLS token and SEP token
        if hasattr(self._vocab, 'cls_token'):
            self._cls_token = self._vocab.cls_token
        else:
            self._cls_token = self._vocab.bos_token
        if hasattr(self._vocab, 'sep_token'):
            self._sep_token = self._vocab.sep_token
        else:
            self._sep_token = self._vocab.eos_token
        self._padding_token = self._vocab.padding_token

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        If vocab.cls_token and vocab.sep_token are not present,
        vocab.bos_token and vocab.eos_token are used instead.

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        # 수정된 부분(https://github.com/SKTBrain/KoBERT/issues/104#issuecomment-1544185148)
        # tokenize를 사용하지 않으면 tokenize와 vocabulary encoding이 같이 일어남
        # 이 경우의 결과가 dictionary이므로, key 값에 대해 encoding이 한 번 더 일어나, 결과가 [2, 0, 0, 0, 3]으로 고정됨
        # 후에 encoding을 한 번 더 하지 않는 방법도 있으나, padding을 위해 초기에 tokenize만 하는 방식으로 수정

        # 변경 전)
        # tokens_a = self._tokenizer(text_a)
        # 변경 후)
        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        tokens.append(self._cls_token)
        tokens.extend(tokens_a)
        tokens.append(self._sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(self._sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._vocab[tokens]

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([self._vocab[self._padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
# BERT data set
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        # 위에서 KoBERT에 맞게 재정의한 BERTSentenceTransform을 사용
        transform = KoBERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        # 각각의 문장과 그 감정의 인덱스를 feature와 label로서 저장
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1

# epoch 수정 파트
num_epochs = 5

max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# BERT 모델, Vocabulary 불러오기

# 변경 전)
# tokenizer = get_tokenizer()
# bertmodel, vocab = get_pytorch_kobert_model()

# 변경 후)
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
# KoBERT에서 제공하는 BERT classifier class
# 참고: https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb

# torch의 nn.Module를 상속(inheritence) 받음
class BERTClassifier(nn.Module):
    def __init__(self,
                        bert,
                        hidden_size = 768,
                        num_classes=2,    # 감정 개수만큼 클래스 수 조정
                        dr_rate=None,
                        params=None):
        # nn.Module의 property를 parameter로 초기화
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    # forward에서 사용할 padding을 구분하는 mask를 생성하는 함수
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    # nn.Module의 forward 함수 override (함수의 parameter는 data loader의 data set 참고)
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        #bert model returns 'last_hidden_state' and 'pooler_output'
        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[1]

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
best_acc=0.0
best_loss=99999999
ckpt_path="/content/drive/" + myDrivePath
ckpt_name=ckpt_path+"saved_model.pt"

In [ ]:
# 모델 불러오기(load)
emotion_array = ['분노', '기쁨', '불안', '당황', '슬픔', '상처']

model = BERTClassifier(bertmodel,  num_classes=len(emotion_array), dr_rate=0.5).to(device)
model.load_state_dict(torch.load(ckpt_name, map_location=device))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
# 모델 검증(inference)
def predict(predict_sentence):
    # 2차원 배열임에 주의
    data_set = [[predict_sentence, '0']]

    # tokenize 및 data_loader 설정
    test_set = BERTDataset(data_set, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=4)

    # model을 evaluate mode로 설정
    model.eval()
    # loss에서의 backward와 같은 gradient 연산을 Tensor에 기록하는 것을 중단(연산 속도 증가)
    with torch.no_grad():
        # iteration은 data set이 1개이므로, 1회만 수행
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            # forward propagation
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)

            # test 결과 저장
            test_eval=[]
            for i in out:
                # 결과를 index로 출력
                logits = i.detach().cpu().numpy()
                # index에 해당하는 감정 저장
                test_eval.append(emotion_array[np.argmax(logits)])

            # test 결과 출력
            print(">> 입력하신 내용에서 " + test_eval[0] + "(이)가 느껴집니다.")

In [ ]:
# TEST
sentences = [
    "교수님 사랑합니다",
    "시험을 망쳤어",
    "친구랑 싸웠어",
    "프로그래밍 좋아",
    "여친과 헤어졌어",
    "면접을 잘 본 것 같아",
    "퀴즈에서 좋은 점수를 얻지 못할까봐 너무나도 불안해",
    "합격자 명단에 내 이름이 없었어",
    "내가 지병이 있다는 이유로 회사에서 불합격 통보를 내렸어"
]

for sentence in sentences:
  print(sentence)
  predict(sentence)

교수님 사랑합니다
>> 입력하신 내용에서 기쁨(이)가 느껴집니다.
시험을 망쳤어
>> 입력하신 내용에서 분노(이)가 느껴집니다.
친구랑 싸웠어
>> 입력하신 내용에서 슬픔(이)가 느껴집니다.
프로그래밍 좋아
>> 입력하신 내용에서 기쁨(이)가 느껴집니다.
여친과 헤어졌어
>> 입력하신 내용에서 슬픔(이)가 느껴집니다.
면접을 잘 본 것 같아
>> 입력하신 내용에서 기쁨(이)가 느껴집니다.
퀴즈에서 좋은 점수를 얻지 못할까봐 너무나도 불안해
>> 입력하신 내용에서 불안(이)가 느껴집니다.
합격자 명단에 내 이름이 없었어
>> 입력하신 내용에서 당황(이)가 느껴집니다.
내가 지병이 있다는 이유로 회사에서 불합격 통보를 내렸어
>> 입력하신 내용에서 분노(이)가 느껴집니다.
